In [ ]:
import re
import json
from pprint import pprint
from random import random

# データの読み込み
他のモデルに追加で書き込む場合は
```py
# model = json.load("model.json")
```
のコメントアウトを外す

In [ ]:
f = open("cmd_data.txt","r")
cmd_datas = [row.strip().split() for row in f]
f.close()

model = {}
# model = json.load("model.json")

def std_val(text):
    return re.sub("[\[\]\(\)]","",text)

In [ ]:
ex_cmd = []

for row in cmd_datas:
    cmd = []
    
    # コマンドのタイプを決定
    for i in range(len(row)):
        if row[i][0] == "-":
            c_type ="option"
        elif row[i][0] == "(":
            c_type = "arg"
        elif row[i][0] == "[":
            c_type = "arg"
        elif row[i][0] == "|":
            c_type = "pipe"
        elif row[i] in ["<","<<",">",">>"]:
            c_type = "redirect"
        elif i == 0 or row[i-1] in ["|"]:
            c_type = "command"
        else:
            c_type = "option"
        
        cmd.append({
            "val":std_val(row[i]),
            "type":c_type
        })
    ex_cmd.append(cmd)


# 共通の説明の定義
パイプ処理やリダイレクトなど、説明が共通なものはここで定義する
```
git rm
rm 
```
など(上の例だとrm)他のコマンドで出現するものは共通の説明に追加しない

In [ ]:
def cmd_print(cmd):
    print(*[v["val"] for v in cmd])

common_dic ={
    "|":"前のコマンドの結果を次に受け渡す",
    ">":"前のコマンドの結果を次のファイルに追加(上書き)",
    ">>":"前のコマンドの結果を次のファイルに追加(追加)",
    "xargs":"前のコマンドの結果からコマンドラインを作成"
}

# モデルの作成
コマンドの各要素に対して表示する説明を入力する

In [ ]:
for cmd in ex_cmd:
    path = model
    for cell in cmd:
        val = cell["val"]
        
        if val in path:
            path[val]["weight"] += 1
        elif "@place" in path and cell["type"] == "arg":
            path["@place"]["weight"] += 1
        else:
            if val in common_dic:
                des = common_dic[val]
            else:
                cmd_print(cmd)
                print("target : ",val)
                des = input()
            if cell["type"] == "arg":
                path["@place"] = {
                    "placeholder":val,
                    "type":cell["type"],
                    "weight":1,
                    "des":des,
                    "@next":{}
                }
            else:
                path[val] = {
                    "val":val,
                    "type":cell["type"],
                    "weight":1,
                    "des":des,
                    "@next":{}
                }    
        if cell["type"] == "arg":
            path = path["@place"]["@next"]
        else:
            try:
                path = path[val]["@next"]
            except:
                pprint(path)
                
        

モデルの保存

In [ ]:
with open("../app/src/renderer/assets/model.json","w") as f:
    json.dump(model,f,indent=2,ensure_ascii=False)